In [ ]:
%matplotlib inline
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import collections
import warnings 
from netCDF4 import default_fillvals
from scipy.stats import hmean

# import soil functions
from parameter_functions import (is_soil_class, is_param_value, classify_soil_texture)

# import veg functions
from parameter_functions import (calculate_cv_pft, map_pft_to_nldas_class, is_overstory, 
                                 calc_root_fract, calc_root_depth_rz1, calc_root_depth_rz2)
# import soil layer aggregation functions 
from parameter_functions import (calculate_first_layer_harmonic_mean, calculate_second_layer_harmonic_mean, 
                                 calculate_third_layer_harmonic_mean, calculate_first_layer_arithmetic_mean, 
                                 calculate_second_layer_arithmetic_mean, calculate_third_layer_arithmetic_mean)

from parameter_functions_v2 import (soil_class_values, calculate_init_moist, 
                                    calculate_baseflow_parameters, create_empty_arrays, 
                                    create_parameter_dataset, calculate_nveg_pfts)

# define fillvals
fillval_f = default_fillvals['f8']
fillval_i = default_fillvals['i4']

This is a notebook for deriving new VIC 5 parameters in the RASM domain at multiple resolutions. 

Currently 50km (`wr50a_ar9v`) and 25km (`wr50a_ar9v`) are supported. 

The accompanying excel sheet `deriving_new_parameters_v2.xlsx` and Word Doc `procedure_for_derivation.docx` provide additional citations and details on methods. 

__Set resolution for filenames__

In [ ]:
res = '50km'

# set file extensions 
if res == "50km":
    grid = 'wr50a_ar9v4'
elif res == "25km":
    grid = 'wr25b_ar9v4'
elif res == "10km":
    raise ValueError("we do not have a domain file at %s so this option has not been implemented" %res)
    
if res == "50km":
    nj = 205
    ni = 275
elif res == "25km":
    nj = 413
    ni = 551
    
num_veg = 17
print("calculating parameters at %s" %res)

In [ ]:
inputdata = '/p/home/gergel/data/inputdata'

__Set domain file__

In [ ]:
if res == '50km':
    domain = xr.open_dataset(os.path.join(inputdata, 
                                      'domain.lnd.wr50a_ar9v4.100920.nc'))
elif res == "25km":
    domain = xr.open_dataset(os.path.join(inputdata, 
                                      'domain.lnd.wr25b_ar9v4.170413.nc'))
masknan_vals = domain['mask'].where(domain['mask'] == 1).values

In [ ]:
old_params = xr.open_dataset(os.path.join(inputdata, 
                                          'vic_params_wr50a_vic5.0.dev_20160328.nc'))

__Options__

In [ ]:
# if set to True, add additional organic_fract options to parameter file, including soil organic fraction, 
# soil particle density of OM, and bulk density of OM 
organic_fract = True
max_snow_albedo = True
bulk_density_comb = True

__Load soil data__

In [ ]:
soil_data_vars = collections.OrderedDict()
soil_data_vars['silt'] = 'silt_sl*'
soil_data_vars['sand'] = 'sand_sl*'
soil_data_vars['clay'] = 'clay_sl*'
soil_data_vars['bulk_density'] = 'bulk_density_sl*'
soil_data_vars['organic_fract'] = 'organic_fract_sl*'

# soil data dict with nlayer = 7 (base resolution of data)
soil_data = {}
soil_data_dir = '/u/home/gergel/data/parameters/soil_data/rasm_grid_netcdfs/%s' %res
for soil_var, soil_wildcard in soil_data_vars.items(): 
    soil_data[soil_var] = xr.open_mfdataset(os.path.join(inputdata, soil_wildcard),
                                            concat_dim='nlayer', 
                                            data_vars='all', 
                                            coords='all')

__calculate soil types based on percent clay, percent sand and bulk density__


In [ ]:
# classify_soil_texture(sand, clay, silt)
soil_type_array = xr.apply_ufunc(classify_soil_texture, 
                                 soil_data['sand']['sand'].where(domain.mask == 1), 
                                 soil_data['clay']['clay'].where(domain.mask == 1),  
                                 soil_data['silt']['silt'].where(domain.mask == 1),
                                 dask='allowed',
                                 vectorize=True)

In [ ]:
ksat = xr.apply_ufunc(soil_class_values, 
                      soil_type_array,
                      'ksat',
                      dask='allowed',
                      vectorize=True)
quartz = xr.apply_ufunc(soil_class_values, 
                        soil_type_array,
                        'quartz',
                        dask='allowed',
                        vectorize=True)
Wcr_FRACT = xr.apply_ufunc(soil_class_values, 
                           soil_type_array,
                           'Wcr_FRACT',
                           dask='allowed',
                           vectorize=True)
Wpwp_FRACT = xr.apply_ufunc(soil_class_values, 
                            soil_type_array,
                           'Wpwp_FRACT',
                            dask='allowed',
                            vectorize=True)
b = xr.apply_ufunc(soil_class_values, 
                   soil_type_array,
                   'b',
                   dask='allowed',
                   vectorize=True)
bulk_density_min = xr.apply_ufunc(soil_class_values, 
                   soil_type_array,
                   'bulk_density',
                   dask='allowed',
                   vectorize=True)
resid_moist = xr.apply_ufunc(soil_class_values, 
                             soil_type_array,
                             'resid_moist',
                             dask='allowed',
                             vectorize=True)

__Load regridded GTOPO 30 data, data var for elevation is called `Band1`, in (m)__

In [ ]:
gtopo_filename = 'sdat_10003_1_20180525_151136146_%s.nc' %grid
gtopo = xr.open_dataset(os.path.join(inputdata, gtopo_filename))
elev = gtopo['Band1']

__Load regridded WORLDCLIM climate data for annual t and p__

In [ ]:
clim_direc = '/u/home/gergel/data/parameters/world_clim_data/%s' %res
prec = xr.open_mfdataset(os.path.join(inputdata, 'prec*'),
                                      concat_dim='time', 
                                      data_vars=['prec'], 
                                      coords='all')

# aggregate to annual, need average annual precip
annual_precip = prec['prec'].sum('time')

temp = xr.open_mfdataset(os.path.join(inputdata, 'tavg*'),
                                      concat_dim='time', 
                                      data_vars='all', 
                                      coords='all')
tavg = temp['tavg'].mean('time')

__Load CLM PFTs to use for vegetation parameters__

In [ ]:
pfts_data_dir = '/u/home/gergel/data/parameters/pfts/regridded_pfts'
pfts_filename = 'mksrf_landuse_rc2000_c110913_%s.nc' %grid
veg_data = xr.open_dataset(os.path.join(inputdata, pfts_filename))

__Calculate Cv from PFTs__

In [ ]:
cv = xr.apply_ufunc(calculate_cv_pft, 
                    veg_data['PCT_PFT'].where(domain.mask == 1),
                    dask='allowed',
                    vectorize=True)

__Calculate number of active PFTs, `Nveg`__ 

In [ ]:
Nveg = xr.apply_ufunc(calculate_nveg_pfts,
                      veg_data['PCT_PFT'].where(domain.mask == 1),
                      dask='allowed',
                      input_core_dims=[['pft']],
                      vectorize=True)

__Load LAI and vegetation height, `MONTHLY_LAI` and `MONTHLY_HEIGHT_TOP`__

In [ ]:
lai_file = xr.open_dataset(os.path.join(inputdata, 
                                   'mksrf_lai_78pfts_simyr2005.c170413_%s_lai.nc' %grid))

In [ ]:
veg_height_file = xr.open_dataset(os.path.join(inputdata, 
                                'mksrf_lai_78pfts_simyr2005.c170413_%s_veg_height.nc' %grid))

LAI and veg_height from CLM and `PCT_PFT` from CLM have a different number of PFTs (`PCT_PFT` has one more PFT, 17 vs 16). The extra PFT in `PCT_PFT` has `PCT_PFT` = 0 over the entire RASM domain, so I just slice the LAI and veg_height from the 0th PFT (water/bare soil) and concatenate it for the 16th PFT. 

In [ ]:
lai_slice = lai_file['MONTHLY_LAI'].isel(pft = 0)
vegheight_slice = veg_height_file['MONTHLY_HEIGHT_TOP'].isel(pft=0)

In [ ]:
lai = xr.concat([lai_file['MONTHLY_LAI'], lai_slice], dim='pft')
veg_height = xr.concat([veg_height_file['MONTHLY_HEIGHT_TOP'], vegheight_slice], dim='pft')

In [ ]:
veg_rough = 0.123 * veg_height
displacement = 0.67 * veg_height

displacement.values[displacement.values == 0] = 1.0

__Change dims and order of dims of LAI array__

In [ ]:
lai = lai.rename({'time': 'month', 'pft': 'veg_class'})
lai = lai.transpose('veg_class', 'month', 'nj', 'ni')

veg_rough = veg_rough.rename({'time': 'month', 'pft': 'veg_class'})
veg_rough = veg_rough.transpose('veg_class', 'month', 'nj', 'ni')

displacement = displacement.rename({'time': 'month', 'pft': 'veg_class'})
displacement = displacement.transpose('veg_class', 'month', 'nj', 'ni')

__Note__: map albedo, root zone fraction and root zone depth based on vegetation type. see `deriving_new_parameters_v2.xlsx` sheet titled `PFT-NLDAS Mapping` for mapping between NLDAS vegetation classes (used in old VIC 5 parameters) and CLM PFTs. This mapping is based on obvious relationships and some approximations (used for PFTs 8-11).

__Create Dataset for variables and define data_vars__

In [ ]:
# loop over pft classes and months 
for pft in veg_data.pft.values:
    for month in old_params.month.values:
        nldas = map_pft_to_nldas_class(pft)
        if nldas == 0 or nldas == 1:
            albedo = 0.12
        elif nldas >= 2 and nldas <= 5:
            albedo = 0.18
        elif nldas >= 6 and nldas <= 8:
            albedo = 0.19
        elif nldas == 9:
            albedo = 0.2
        elif nldas == 10: 
            albedo = 0.12
        elif nldas == 11: 
            albedo = 0.2
        params['albedo'].values[pft, month-1, :, :] = np.ones((1, 1, nj, ni)) * albedo

__load hydroclimate classes__

In [ ]:
hydro_classes = xr.open_dataset(os.path.join(inputdata,
                                             'hydroclimate_masks_%s.nc' %grid))

__baseflow parameters: Ds, Dsmax, Ws__

In [ ]:
d1 = calculate_baseflow_parameters(domain, inputdata, hydro_classes, "d1")
params['Ds'].values = d1

d2 = calculate_baseflow_parameters(domain, inputdata, hydro_classes, "d2")
params['Dsmax'].values = d2

d3 = calculate_baseflow_parameters(domain, inputdata, hydro_classes, "d3")
params['Ws'].values = d3

d4 = calculate_baseflow_parameters(domain, inputdata, hydro_classes, "d4")
params['c'].values = d4

__b_i (`infilt`)__

First set of changes: 
--cold/WDS/WS no permafrost: change from 0.05 to 0.25 
--cold/WDS/CS no permafrost: change from 0.5 to 0.25 
--polar: change from 0.1 to 0.35


In [ ]:
bi = np.copy(masknan_vals)
bi[np.nonzero(hydro_classes['arid'].values)] = 0.05
bi[np.nonzero(hydro_classes['temperate_dry'].values)] = 0.05
bi[np.nonzero(hydro_classes['cold_dry_perma'].values)] = 0.3
bi[np.nonzero(hydro_classes['cold_dry_noperma'].values)] = 0.5
bi[np.nonzero(hydro_classes['cold_wds_ws_perma'].values)] = 0.3
######### CHANGED ##############################
bi[np.nonzero(hydro_classes['cold_wds_ws_noperma'].values)] = 0.05
# bi[np.nonzero(hydro_classes['cold_wds_ws_noperma'].values)] = 0.25
bi[np.nonzero(hydro_classes['cold_wds_cs_perma'].values)] = 0.3
######### CHANGED ##############################
bi[np.nonzero(hydro_classes['cold_wds_cs_noperma'].values)] = 0.5
# bi[np.nonzero(hydro_classes['cold_wds_cs_noperma'].values)] = 0.25
######### CHANGED ##############################
bi[np.nonzero(hydro_classes['polar'].values)] = 0.1
# bi[np.nonzero(hydro_classes['polar'].values)] = 0.35

params['infilt'].values = bi

__soil depths (`depth`)__

In [ ]:
D1 = np.copy(masknan_vals)
D2 = np.copy(masknan_vals)
D3 = np.copy(masknan_vals)
D1[np.nonzero(domain.mask.values)] = 0.3
D3[np.nonzero(domain.mask.values)] = 0.5

D2[np.nonzero(hydro_classes['arid'].values)] = 2.0
D2[np.nonzero(hydro_classes['temperate_dry'].values)] = 2.0
D2[np.nonzero(hydro_classes['cold_dry_perma'].values)] = 0.5
D2[np.nonzero(hydro_classes['cold_dry_noperma'].values)] = 0.5
D2[np.nonzero(hydro_classes['cold_wds_ws_perma'].values)] = 2.0
######### CHANGED ##############################
# D2[np.nonzero(hydro_classes['cold_wds_ws_noperma'].values)] = 0.5
D2[np.nonzero(hydro_classes['cold_wds_ws_noperma'].values)] = 2.0
######### CHANGED ##############################
# D2[np.nonzero(hydro_classes['cold_wds_cs_perma'].values)] = 1.1
D2[np.nonzero(hydro_classes['cold_wds_cs_perma'].values)] = 0.5
######### CHANGED ##############################
# D2[np.nonzero(hydro_classes['cold_wds_cs_noperma'].values)] = 0.3
D2[np.nonzero(hydro_classes['cold_wds_cs_noperma'].values)] = 2.0
######### CHANGED ##############################
# D2[np.nonzero(hydro_classes['polar'].values)] = 0.3
D2[np.nonzero(hydro_classes['polar'].values)] = 0.5

depths = np.rollaxis(np.dstack((D1, D2, D3)), axis=2)
params['depth'].values = depths

__aggregate ISRIC soil data to VIC soil depths__

first need array of soil depths 

In [ ]:
soil_depths = params['depth'].sum(axis=0)
print("max soil depth is %.1f m" % soil_depths.max())

In [ ]:
ksat_l1 = xr.apply_ufunc(calculate_first_layer_harmonic_mean,
                         ksat.isel(nlayer=0), 
                         ksat.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

ksat_l2 = xr.apply_ufunc(calculate_second_layer_harmonic_mean,
                         ksat.isel(nlayer=2), 
                         ksat.isel(nlayer=3),
                         ksat.isel(nlayer=4),
                         ksat.isel(nlayer=5),
                         ksat.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

ksat_l3 = xr.apply_ufunc(calculate_second_layer_harmonic_mean,
                         ksat.isel(nlayer=2), 
                         ksat.isel(nlayer=3),
                         ksat.isel(nlayer=4),
                         ksat.isel(nlayer=5),
                         ksat.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

ksat_vals = np.rollaxis(np.dstack((ksat_l1, ksat_l2, ksat_l3)), 
                        axis=2)

params['Ksat'].values = ksat_vals

In [ ]:
# bulk_density
bdm_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         bulk_density_min.isel(nlayer=0), 
                         bulk_density_min.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

bdm_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         bulk_density_min.isel(nlayer=2), 
                         bulk_density_min.isel(nlayer=3),
                         bulk_density_min.isel(nlayer=4),
                         bulk_density_min.isel(nlayer=5),
                         bulk_density_min.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

bdm_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         bulk_density_min.isel(nlayer=2), 
                         bulk_density_min.isel(nlayer=3),
                         bulk_density_min.isel(nlayer=4),
                         bulk_density_min.isel(nlayer=5),
                         bulk_density_min.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

bdm_vals = np.rollaxis(np.dstack((bdm_l1, bdm_l2, bdm_l3)), 
                        axis=2)
params['bulk_density'].values = bdm_vals

In [ ]:
# expt
b_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         b.isel(nlayer=0), 
                         b.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

b_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         b.isel(nlayer=2), 
                         b.isel(nlayer=3),
                         b.isel(nlayer=4),
                         b.isel(nlayer=5),
                         b.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

b_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         b.isel(nlayer=2), 
                         b.isel(nlayer=3),
                         b.isel(nlayer=4),
                         b.isel(nlayer=5),
                         b.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

expt_vals = np.rollaxis(np.dstack(((b_l1 * 2) + 3, (b_l2 * 2) + 3, (b_l3 * 2) + 3)), 
                        axis=2)
params['expt'].values = expt_vals
params['bubble'].values = (np.copy(params['expt'].values) * 0.32) + 4.3

In [ ]:
# resid_moist
rm_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         resid_moist.isel(nlayer=0), 
                         resid_moist.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

rm_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         resid_moist.isel(nlayer=2), 
                         resid_moist.isel(nlayer=3),
                         resid_moist.isel(nlayer=4),
                         resid_moist.isel(nlayer=5),
                         resid_moist.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

rm_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         resid_moist.isel(nlayer=2), 
                         resid_moist.isel(nlayer=3),
                         resid_moist.isel(nlayer=4),
                         resid_moist.isel(nlayer=5),
                         resid_moist.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)
rm_vals = np.rollaxis(np.dstack((rm_l1, rm_l2, rm_l3)), 
                        axis=2)
params['resid_moist'].values = rm_vals

In [ ]:
# Wcr_FRACT
wcr_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         Wcr_FRACT.isel(nlayer=0), 
                         Wcr_FRACT.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

wcr_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         Wcr_FRACT.isel(nlayer=2), 
                         Wcr_FRACT.isel(nlayer=3),
                         Wcr_FRACT.isel(nlayer=4),
                         Wcr_FRACT.isel(nlayer=5),
                         Wcr_FRACT.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

wcr_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         Wcr_FRACT.isel(nlayer=2), 
                         Wcr_FRACT.isel(nlayer=3),
                         Wcr_FRACT.isel(nlayer=4),
                         Wcr_FRACT.isel(nlayer=5),
                         Wcr_FRACT.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)
wcr_vals = np.rollaxis(np.dstack((wcr_l1, wcr_l2, wcr_l3)), 
                        axis=2)

In [ ]:
# Wpwp_FRACT
wpwp_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         Wpwp_FRACT.isel(nlayer=0), 
                         Wpwp_FRACT.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

wpwp_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         Wpwp_FRACT.isel(nlayer=2), 
                         Wpwp_FRACT.isel(nlayer=3),
                         Wpwp_FRACT.isel(nlayer=4),
                         Wpwp_FRACT.isel(nlayer=5),
                         Wpwp_FRACT.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

wpwp_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         Wpwp_FRACT.isel(nlayer=2), 
                         Wpwp_FRACT.isel(nlayer=3),
                         Wpwp_FRACT.isel(nlayer=4),
                         Wpwp_FRACT.isel(nlayer=5),
                         Wpwp_FRACT.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

wpwp_vals = np.rollaxis(np.dstack((wpwp_l1, wpwp_l2, wpwp_l3)), 
                        axis=2)

In [ ]:
# quartz
qz_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         quartz.isel(nlayer=0), 
                         quartz.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

qz_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         quartz.isel(nlayer=2), 
                         quartz.isel(nlayer=3),
                         quartz.isel(nlayer=4),
                         quartz.isel(nlayer=5),
                         quartz.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

qz_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         quartz.isel(nlayer=2), 
                         quartz.isel(nlayer=3),
                         quartz.isel(nlayer=4),
                         quartz.isel(nlayer=5),
                         quartz.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)
qz_vals = np.rollaxis(np.dstack((qz_l1, qz_l2, qz_l3)), 
                        axis=2)
params['quartz'].values = qz_vals

In [ ]:
if bulk_density_comb == True:
    # bulk_density
    bd_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=0), 
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=1),
                             dask='allowed',
                             vectorize=True)

    bd_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=2), 
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=3),
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=4),
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=5),
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=6),
                             soil_depths,
                             dask='allowed',
                             vectorize=True)

    bd_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=2), 
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=3),
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=4),
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=5),
                             soil_data['bulk_density']['bulk_density'].isel(nlayer=6),
                             soil_depths,
                             dask='allowed',
                             vectorize=True)
    bd_vals = np.rollaxis(np.dstack((bd_l1, bd_l2, bd_l3)), 
                            axis=2)
    params['bulk_density_comb'].values = bd_vals

In [ ]:
if organic_fract == True:
    # organic fract
    of_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=0), 
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=1),
                             dask='allowed',
                             vectorize=True)

    of_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=2), 
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=3),
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=4),
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=5),
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=6),
                             soil_depths,
                             dask='allowed',
                             vectorize=True)

    of_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=2), 
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=3),
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=4),
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=5),
                             soil_data['organic_fract']['organic_fract'].isel(nlayer=6),
                             soil_depths,
                             dask='allowed',
                             vectorize=True)
    of_vals = np.rollaxis(np.dstack(((of_l1/1000), (of_l2/1000), (of_l3/1000))), 
                            axis=2)
    params['organic'].values = of_vals

__calculate porosity from bulk density and soil density__

In [ ]:
sd_l1 = np.copy(masknan_vals)
sd_l2 = np.copy(masknan_vals)
sd_l3 = np.copy(masknan_vals)

sd_l1[np.nonzero(masknan_vals)] = 2685.0
sd_l2[np.nonzero(masknan_vals)] = 2685.0
sd_l3[np.nonzero(masknan_vals)] = 2685.0

sd_vals = np.rollaxis(np.dstack((sd_l1, sd_l2, sd_l3)), 
                        axis=2)
params['soil_density'].values = sd_vals

In [ ]:
if organic_fract == True:
    sd_org_l1 = np.copy(masknan_vals)
    sd_org_l2 = np.copy(masknan_vals)
    sd_org_l3 = np.copy(masknan_vals)

    sd_org_l1[np.nonzero(masknan_vals)] = 1300.0
    sd_org_l2[np.nonzero(masknan_vals)] = 1300.0
    sd_org_l3[np.nonzero(masknan_vals)] = 1300.0

    sd_org_vals = np.rollaxis(np.dstack((sd_org_l1, sd_org_l2, sd_org_l3)), 
                            axis=2)
    params['soil_density_org'].values = sd_org_vals

In [ ]:
# calculate porosity
if bulk_density_comb == True:
    porosity = 1 - (params['bulk_density_comb'] / params['soil_density'])
else:
    porosity = 1 - (params['bulk_density'] / params['soil_density'])

In [ ]:
params['Wpwp_FRACT'].values = wpwp_vals / porosity.values
params['Wcr_FRACT'].values = wcr_vals / porosity.values

__make initial moisture fully saturated__

In [ ]:
init_moist_l1 = xr.apply_ufunc(calculate_init_moist,
                               porosity.isel(nlayer=0), 
                               params.depth.isel(nlayer=0),
                               dask='allowed', 
                               vectorize=True)
init_moist_l2 = xr.apply_ufunc(calculate_init_moist,
                               porosity.isel(nlayer=1), 
                               params.depth.isel(nlayer=1),
                               dask='allowed', 
                               vectorize=True)
init_moist_l3 = xr.apply_ufunc(calculate_init_moist,
                               porosity.isel(nlayer=1), 
                               params.depth.isel(nlayer=2),
                               dask='allowed', 
                               vectorize=True)
init_moist_vals = np.rollaxis(np.dstack((init_moist_l1, init_moist_l2, init_moist_l3)), 
                        axis=2)
params['init_moist'].values = init_moist_vals

__add `off_gmt`__

define `trunk_ratio`, `rarc`, `rmin`, `wind_h`, `RGL`, `rad_atten`, `wind_atten`, `overstory`, `max_snow_albedo`

In [ ]:
# trunk ratio, rarc, rad_atten
trunk_ratio = np.copy(arr_veg_classes)
params['trunk_ratio'].values = trunk_ratio * 0.2
# adjust for bare soil 
params['trunk_ratio'].values[0, :, :] = 0.0

rarc = np.copy(arr_veg_classes)
params['rarc'].values = rarc * 60
# adjust for bare soil
params['rarc'].values[0, :, :] = 100

rad_atten = np.copy(arr_veg_classes)
params['rad_atten'].values = rad_atten * 0.5
# adjust for bare soil 
params['rad_atten'].values[0, :, :] = 0.0

wind_atten = np.copy(arr_veg_classes)
params['wind_atten'].values = wind_atten * 0.5
# adjust for bare soil 
params['wind_atten'].values[0, :, :] = 0.0

In [ ]:
if max_snow_albedo == True:
    # max_albedo
    for pft in veg_data.pft.values:
        # get nldas mapping from pft
        nldas = map_pft_to_nldas_class(pft)
        if nldas == 0:
            max_alb = 0.34
        elif nldas == 1:
            max_alb = 0.37
        elif nldas == 2:
            max_alb = 0.35
        elif nldas == 3: 
            max_alb = 0.35
        elif nldas == 4: 
            max_alb = 0.44
        elif nldas == 5:
            max_alb = 0.69
        elif nldas == 6:
            max_alb = 0.43
        elif nldas == 7:
            max_alb = 0.56
        elif nldas == 8:
            max_alb = 0.70
        elif nldas == 9:
            max_alb = 0.65
        elif nldas == 10:
            max_alb = 0.46
        elif nldas == 11:
            max_alb = 0.84
        params['max_snow_albedo'].values[pft, :, :] = np.ones((1, nj, ni)) * max_alb

In [ ]:
# rmin, wind_h
for pft in veg_data.pft.values:
    # get nldas mapping from pft
    nldas = map_pft_to_nldas_class(pft)
    if nldas >= 0 and nldas <= 3:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=0).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=0).mean())
    elif nldas == 4:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=4).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=4).mean())
    elif nldas >= 5 and nldas <= 6:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=5).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=5).mean())
    elif nldas >= 7 and nldas <= 8:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=7).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=7).mean())
    elif nldas == 9:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=9).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=9).mean())
    elif nldas == 10:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=10).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=10).mean())
    elif nldas == 11:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=11).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=11).mean())
    params['rmin'].values[pft, :, :] = np.ones((1, nj, ni)) * rmin
    params['wind_h'].values[pft, :, :] = np.ones((1, nj, ni)) * wind_h

In [ ]:
# RGL
for pft in veg_data.pft.values:
    # get nldas mapping from pft
    nldas = map_pft_to_nldas_class(pft)
    if nldas >= 0 and nldas <= 3:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=0).mean())
    elif nldas >= 4 and nldas <= 5:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=4).mean())
    elif nldas >= 6 and nldas <= 8:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=6).mean())
    elif nldas >= 9 and nldas <= 10:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=9).mean())
    elif nldas == 11:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=11).mean())
    params['RGL'].values[pft, :, :] = np.ones((1, nj, ni)) * rgl

In [ ]:
# overstory
overstory = np.copy(arr_veg_classes)
for pft in veg_data.pft.values:
    nldas = map_pft_to_nldas_class(pft)
    if nldas > 6:
        # no overstory
        overstory = 0.0
    else: 
        overstory = 1.0
    params['overstory'].values[pft, :, :] = overstory

In [ ]:
root_depth_rz1 = xr.apply_ufunc(calc_root_depth_rz1,
                           params['Cv'].where(domain.mask == 1), 
                           dask='allowed',
                           vectorize=True)
root_depth_rz2 = xr.apply_ufunc(calc_root_depth_rz2,
                           params['Cv'].where(domain.mask == 1), 
                           dask='allowed',
                           vectorize=True)
root_depth = xr.concat([root_depth_rz1, root_depth_rz2],
                      dim='root_zone').transpose('veg_class', 'root_zone', 'nj', 'ni')

In [ ]:
# root fract 

rz = 0
for pft in veg_data.pft.values:
    if pft == 0:
        root_fract_rz1 = xr.apply_ufunc(calc_root_fract,
                                        params['Cv'].isel(veg_class=pft),
                                        str(pft),
                                        str(rz),
                                        dask='allowed',
                                        vectorize=True)
    else: 
        root_fract_rz1 = xr.concat([root_fract_rz1, xr.apply_ufunc(calc_root_fract,
                                                                   params['Cv'].isel(veg_class=pft),
                                                                   str(pft),
                                                                   str(rz),
                                                                   dask='allowed',
                                                                   vectorize=True)],
                                  dim='veg_class')
rz = 1
for pft in veg_data.pft.values:
    if pft == 0:
        root_fract_rz2 = xr.apply_ufunc(calc_root_fract,
                                        params['Cv'].isel(veg_class=pft), 
                                        str(pft),
                                        str(rz),
                                        dask='allowed',
                                        vectorize=True)
    else: 
        root_fract_rz2 = xr.concat([root_fract_rz2, xr.apply_ufunc(calc_root_fract,
                                                                   params['Cv'].isel(veg_class=pft), 
                                                                   str(pft),
                                                                   str(rz),
                                                                   dask='allowed',
                                                                   vectorize=True)],
                                  dim='veg_class')
        
root_fract = xr.concat([root_fract_rz1, root_fract_rz2], dim='root_zone').transpose('veg_class', 'root_zone', 'nj', 'ni')

In [ ]:
params['root_depth'].values = root_depth
params['root_fract'].values = root_fract

__albedo__

In [ ]:
if res == "50km":
    params['off_gmt'].values = old_params['off_gmt'].values

elif res == "25km":
    # load regridded off_gmt from 50km to 25km 
    off_gmt = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters', 
                                           'off_gmt_wr25b_ar9v4.nc'))
    params['off_gmt'].values = off_gmt['off_gmt'].values

__add: elev, c, phi_s, avg_T, dp, bubble, soil_density, off_gmt, rough, snow_rough,
annual_prec, fs_active__